In [9]:
import os
import joblib
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [10]:
# Create directories for saving artifacts
os.makedirs("models", exist_ok=True)
os.makedirs("forecasts", exist_ok=True)

In [11]:
# ======================
# 1. Fetch Historical Data
# ======================
assets = ['TSLA', 'BND', 'SPY']
print("Downloading data from Yahoo Finance...")
data = {}
for asset in assets:
    df = yf.download(asset, start="2015-01-01", end="2025-01-31")
    data[asset] = df['Close']  # Extract closing prices
    print(f"{asset} data shape: {data[asset].shape}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

TSLA data shape: (2535, 1)



[*********************100%***********************]  1 of 1 completed

BND data shape: (2535, 1)
SPY data shape: (2535, 1)


In [12]:
# ======================
# 2. Preprocess Data
# ======================
def preprocess_series(series):
    """Handle missing values and set frequency"""
    series = series.asfreq('B')  # Business days
    series = series.ffill()      # Forward-fill missing values
    series.index.freq = 'B'      # Explicitly set frequency
    return series

print("\nPreprocessing data...")
for asset in assets:
    data[asset] = preprocess_series(data[asset])
    print(f"{asset} missing values after processing: {data[asset].isnull().sum()}")


Preprocessing data...
TSLA missing values after processing: Ticker
TSLA    0
dtype: int64
BND missing values after processing: Ticker
BND    0
dtype: int64
SPY missing values after processing: Ticker
SPY    0
dtype: int64


In [13]:
# 3. Train SARIMA Models
# ======================
def train_sarima(series, order=(4,1,0), seasonal_order=(1,1,1,5)):
    """Train and save SARIMA model"""
    model = SARIMAX(series,
                   order=order,
                   seasonal_order=seasonal_order,
                   enforce_stationarity=False)
    model_fit = model.fit(disp=False)
    return model_fit

print("\nTraining models...")
models = {}
for asset in assets:
    print(f"Training {asset} model...")
    models[asset] = train_sarima(data[asset])
    joblib.dump(models[asset], f"models/{asset}_sarima.pkl")
    print(f"{asset} model AIC: {models[asset].aic:.2f}")


Training models...
Training TSLA model...
TSLA model AIC: 17032.24
Training BND model...
BND model AIC: -134.00
Training SPY model...
SPY model AIC: 13949.54
